# Build Image

In this notebook, we show:

- Create/Register a Docker image in ACR using AzureML


In [48]:
import docker
#import numpy as np
import requests
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azureml._model_management._util import (get_docker_client, pull_docker_image)
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.image import ContainerImage
from dotenv import find_dotenv, get_key, set_key
from testing_utilities import to_img, plot_predictions, get_auth, wait_until_ready


In [49]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [50]:
resource_group = get_key(env_path, 'resource_group')
model_name = 'maskrcnn_resnet50_model'
image_name = get_key(env_path, 'image_name')

## Get workspace
Load existing workspace from the config file info.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create Image

In [52]:
# create yml file to be used in the image
conda_pack = []
requirements = ["torch==1.1.0","torchvision==0.3","Pillow==5.2.0", "azureml-defaults", "azureml-contrib-services", "toolz==0.9.0"]

imgenv = CondaDependencies.create(conda_packages=conda_pack,pip_packages=requirements)
with open("img_env.yml", "w") as f:
    f.write(imgenv.serialize_to_string())

In [53]:

image_config = ContainerImage.image_configuration(execution_script = "driver.py",
                                                  runtime = "python",
                                                  conda_file = "img_env.yml",
                                                  description = "Image for torchvision MaskRCNN ResNet 50 Model",
                                                  tags = {"name":"object detection","project":"AzureML"}, 
                                                  #dependencies = ["resnet152.py"],
                                                  enable_gpu = True
                                                 )


In [54]:
# create image. It may take upto 15-20 minutes. 
image = ContainerImage.create(name = image_name,
                              # this is the model object
                              models = [ws.models[model_name]],                              
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running..........................................................................................
Succeeded
Image creation operation finished for image imgformlmodel:7, operation "Succeeded"


In [55]:
# You can find the logs of image creation
# image.image_build_log_uri

# You can get the image object when not creating a new image
# image = ws.images['image1']

In [ ]:
# Getting your container details
ws = Workspace.from_config(auth=get_auth())
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id

client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value
print('ContainerURL:{}'.format(image.image_location))
print('Servername: {}'.format(reg_name))
print('Username: {}'.format(username))
print('Password: {}'.format(password))

In [ ]:
acr_name = reg_name # azure container registry  name. e.g. arc_name = "myacr"
acr_login_server = '{}.azurecr.io'.format(acr_name)
set_key(env_path,"acr_name", acr_name)
set_key(env_path,"acr_login_server", acr_login_server)
set_key(env_path,"acr_password", password)

In [ ]:
#img_model_location = '{}/{}:1'.format(acr_login_server, image_name)
ml_img_location = image.image_location
set_key(env_path,"ml_img_location", ml_img_location)

Next we will proceed with notebook [04_DeployOnIOTedge.ipynb](04_DeployOnIOTedge.ipynb.ipynb).